### Exemples de query possible sur DuckDB pour de l'analyses

#### Pour acceder aux données de DuckDB en local depuis le volume montée :
kubectl apply -f jupyter-pod.yaml
kubectl port-forward pod/jupyter-duckdb 8888:8888 -n airflow
#### Pour choper le token :
kubectl logs pod/jupyter-duckdb -n airflow

In [3]:
import duckdb

# Connexion au fichier DuckDB monté dans le pod
conn = duckdb.connect("/mnt/duckdb/analytics.duckdb")

IOException: IO Error: Cannot open file "/mnt/duckdb/analytics.duckdb": No such file or directory

In [ ]:
# Liste des tables disponibles
conn.sql("SHOW TABLES").df()

1 — Afficher les 10 premières lignes de gold_notif_impact_per_day

In [ ]:
conn.sql("SELECT * FROM gold_notif_impact_per_day LIMIT 10").df()

2 — Durée moyenne de session après notification

In [ ]:
conn.sql("""
    SELECT 
        notif_date, 
        AVG(time_spend_after_success) AS avg_time_spent
    FROM gold_notif_impact_per_day
    GROUP BY notif_date
    ORDER BY notif_date DESC
""").df()

3) Top contenus de notifications les plus engageants

In [ ]:
conn.sql("""
    SELECT 
        content_notif,
        COUNT(*) AS nb_notifs,
        AVG(time_spend_after_success) AS avg_time
    FROM gold_notif_impact_per_day
    GROUP BY content_notif
    ORDER BY avg_time DESC
    LIMIT 10
""").df()

4) Taux de succès de notifications

In [ ]:
conn.sql("""
    SELECT 
        notif_date,
        SUM(CASE WHEN is_success THEN 1 ELSE 0 END) AS seen,
        COUNT(*) AS total,
        ROUND(100.0 * SUM(CASE WHEN is_success THEN 1 ELSE 0 END) / COUNT(*), 2) AS success_rate
    FROM gold_notif_impact_per_day
    GROUP BY notif_date
    ORDER BY notif_date
""").df()
